In [48]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from tqdm import tqdm
import zipfile
import io
import os

In [39]:
def request_area_info():
    url = "https://www.jma.go.jp/jma/kishou/know/amedas/ame_master.zip"
    response = requests.get(url)
    response.raise_for_status()
    #ID_info = response.json()

    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # extract  csv file
        csv_name = next(name for name in z.namelist() if name.lower().endswith('.csv'))

        with z.open(csv_name) as f:
            print(csv_name)
            ID_info_df = pd.read_csv(f, encoding='cp932')

    #ID_info_df = pd.DataFrame.from_dict(ID_info, orient='index')

    # add ID column
    ID_info_df["ID"] = ID_info_df["観測所番号"].astype(int)
    ID_info_df = ID_info_df.reset_index()
    return ID_info_df

In [40]:
def request_one_day_amedas_data(date):
    URL = "https://www.jma.go.jp/bosai/amedas/data/map/"
    date = datetime.strptime(date, "%Y%m%d")
    one_hour_data = []

    for hour in tqdm(range(24)):
        timestamp = date.strftime(f"%Y%m%d{hour:02d}0000")
        url = f"{URL}{timestamp}.json"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame.from_dict(data, orient='index')
   
            df['date'] = date.strftime('%Y-%m-%d')
            df['hour'] = f"{hour:02d}:00:00"
                
            one_hour_data.append(df)

            # Sleep to avoid overwhelming the server
            time.sleep(0.5)
        else:
            print(f"failed to requrst data at {timestamp}")
            

    one_day_df = pd.concat(one_hour_data)

    # add ID column
    one_day_df["ID"] = one_day_df.index.astype(int)
    one_day_df = one_day_df.reset_index()
    
    return one_day_df

In [55]:
def import_raw_data(date):
    # request amedas data
    print("data request start!")
    main_df = request_one_day_amedas_data(date)
    ID_info_df = request_area_info()
    print("data request end!")

    # merge 
    main_df = pd.merge(main_df, ID_info_df, on="ID", how="left")

    # save
    data_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    save_path = os.path.join(data_dir, "data", f"amedas_data_{date}.csv")
    main_df.to_csv(save_path, index=False)

In [56]:
# run 
DATE = "20250713"  
df = import_raw_data(DATE)

data request start!


100%|██████████| 24/24 [00:15<00:00,  1.51it/s]


ame_master_20250707.csv
data request end!
